In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

import gc
import pickle
from collections import OrderedDict
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import time

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)

yrs_range= '2002_2020'
em = 'bc006'

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr200601.nc')[0][1,:,:]

for yr in range(2001,20010):
    print(yr)
    ######################################################
    ### Get all the data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}/WholeYear/cube_{yr}.pkl"

    if os.path.exists(pickle_file_filepath):
        print("Pickle file exists, so loading that")
        full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    else:
        print("Pickle file doesnt exist, so creating and then saving that")

        ### Get the data filepaths
        print(f"Loading data for year {yr}")

        # Create cube list
        cubes = load_files_to_cubelist(yr, general_filename)

        # Join them into one (with error handling to deal with times which are wrong)
        try:
            full_year_cube = cubes.concatenate_cube()
            print("Concatenation successful!")
        except Exception as e:
            print(f"Initial concatenation failed: {str(e)}")

            # If initial concatenation fails, remove problematic cubes and try again
            try:
                full_year_cube = remove_problematic_cubes(cubes)
                print("Concatenation successful after removing problematic cubes!")
            except RuntimeError as e:
                print(f"Concatenation failed after removing problematic cubes: {str(e)}")               
        save_cube_as_pickle_file(full_year_cube, pickle_file_filepath)
        
        
    gauge_nums = range(0,1294)
    # Function to process each gauge
    for gauge_num in gauge_nums:
        if not gauge_num in [423, 444, 827, 888]:
                print(f"year is {yr}, gauge num is {gauge_num}")             

                base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear"
                # Create the directory if it doesnt exist
                if not os.path.isdir(base_dir):
                    os.makedirs(base_dir)

                ######################################################
                ## Check if any files are missing, across the 3 filtering options
                # If there are: code will continue to run
                # If not: code will move to next gauge
                ######################################################                
                # Create a flag to record whether we are missing any of the files we need
                missing_files = False

                # Check if we are missing any of the files, and if so, change the flag to True
                if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                    missing_files = True

                # If we are missing some files then get the data for the grid cell, 
                if missing_files:
                    # Find location
                    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

                    # Extract data for the specified indices
                    start= time.time()
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]
                    data = one_location_cube.data
                    end=time.time()
                    print(f"Time to load data is {round(end-start,2)} seconds")

                    ##### Filter cube according to different options
                    # Convert to dataframe
                    df = pd.DataFrame(data, columns=['precipitation (mm/hr)'])
                    df['times'] = one_location_cube.coord('time').units.num2date(one_location_cube.coord('time').points)
                    df['precipitation (mm)'] = df['precipitation (mm/hr)'] / 2   

                    # Search dataframe for events corresponding to durations
                    for duration in [0.5, 1, 2, 3, 6, 12, 24]:

                        filename =  f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"
                        if not os.path.exists(filename):
                            print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr}")
                            # Find events
                            events_v2 = search_for_valid_events(df, duration=duration, Tb0=Tb0)

                            # Save events to CSV
                            for num, event in enumerate(events_v2):
                                if len(event) > 1:
                                        event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")
                                        if event['precipitation (mm/hr)'].isna().any():
                                            print("NANs in this event")
                        else:
                            # print(f"already exists{filename}")
                            pass   
                else:
                    pass
                    # print("already exist")



2001
Pickle file exists, so loading that
year is 2001, gauge num is 0
year is 2001, gauge num is 1
year is 2001, gauge num is 2
year is 2001, gauge num is 3
year is 2001, gauge num is 4
year is 2001, gauge num is 5
year is 2001, gauge num is 6
year is 2001, gauge num is 7
year is 2001, gauge num is 8
year is 2001, gauge num is 9
year is 2001, gauge num is 10
year is 2001, gauge num is 11
year is 2001, gauge num is 12
year is 2001, gauge num is 13
year is 2001, gauge num is 14
year is 2001, gauge num is 15
year is 2001, gauge num is 16
year is 2001, gauge num is 17
year is 2001, gauge num is 18
year is 2001, gauge num is 19
year is 2001, gauge num is 20
year is 2001, gauge num is 21
year is 2001, gauge num is 22
year is 2001, gauge num is 23
year is 2001, gauge num is 24
year is 2001, gauge num is 25
year is 2001, gauge num is 26
year is 2001, gauge num is 27
year is 2001, gauge num is 28
year is 2001, gauge num is 29
year is 2001, gauge num is 30
year is 2001, gauge num is 31
year is 2

year is 2001, gauge num is 385
year is 2001, gauge num is 386
year is 2001, gauge num is 387
year is 2001, gauge num is 388
year is 2001, gauge num is 389
year is 2001, gauge num is 390
year is 2001, gauge num is 391
year is 2001, gauge num is 392
year is 2001, gauge num is 393
year is 2001, gauge num is 394
year is 2001, gauge num is 395
year is 2001, gauge num is 396
year is 2001, gauge num is 397
year is 2001, gauge num is 398
year is 2001, gauge num is 399
year is 2001, gauge num is 400
year is 2001, gauge num is 401
year is 2001, gauge num is 402
year is 2001, gauge num is 403
year is 2001, gauge num is 404
year is 2001, gauge num is 405
year is 2001, gauge num is 406
year is 2001, gauge num is 407
year is 2001, gauge num is 408
year is 2001, gauge num is 409
year is 2001, gauge num is 410
year is 2001, gauge num is 411
year is 2001, gauge num is 412
year is 2001, gauge num is 413
year is 2001, gauge num is 414
year is 2001, gauge num is 415
year is 2001, gauge num is 416
year is 

year is 2001, gauge num is 669
year is 2001, gauge num is 670
year is 2001, gauge num is 671
year is 2001, gauge num is 672
year is 2001, gauge num is 673
year is 2001, gauge num is 674
year is 2001, gauge num is 675
year is 2001, gauge num is 676
year is 2001, gauge num is 677
year is 2001, gauge num is 678
year is 2001, gauge num is 679
year is 2001, gauge num is 680
year is 2001, gauge num is 681
year is 2001, gauge num is 682
year is 2001, gauge num is 683
year is 2001, gauge num is 684
year is 2001, gauge num is 685
year is 2001, gauge num is 686
year is 2001, gauge num is 687
year is 2001, gauge num is 688
year is 2001, gauge num is 689
year is 2001, gauge num is 690
year is 2001, gauge num is 691
year is 2001, gauge num is 692
year is 2001, gauge num is 693
year is 2001, gauge num is 694
year is 2001, gauge num is 695
year is 2001, gauge num is 696
year is 2001, gauge num is 697
year is 2001, gauge num is 698
year is 2001, gauge num is 699
year is 2001, gauge num is 700
year is 

year is 2001, gauge num is 945
year is 2001, gauge num is 946
year is 2001, gauge num is 947
year is 2001, gauge num is 948
year is 2001, gauge num is 949
year is 2001, gauge num is 950
year is 2001, gauge num is 951
year is 2001, gauge num is 952
year is 2001, gauge num is 953
year is 2001, gauge num is 954
year is 2001, gauge num is 955
year is 2001, gauge num is 956
year is 2001, gauge num is 957
year is 2001, gauge num is 958
year is 2001, gauge num is 959
year is 2001, gauge num is 960
year is 2001, gauge num is 961
year is 2001, gauge num is 962
year is 2001, gauge num is 963
year is 2001, gauge num is 964
year is 2001, gauge num is 965
year is 2001, gauge num is 966
year is 2001, gauge num is 967
year is 2001, gauge num is 968
year is 2001, gauge num is 969
year is 2001, gauge num is 970
year is 2001, gauge num is 971
year is 2001, gauge num is 972
year is 2001, gauge num is 973
year is 2001, gauge num is 974
year is 2001, gauge num is 975
year is 2001, gauge num is 976
year is 

year is 2001, gauge num is 1250
year is 2001, gauge num is 1251
year is 2001, gauge num is 1252
year is 2001, gauge num is 1253
year is 2001, gauge num is 1254
year is 2001, gauge num is 1255
year is 2001, gauge num is 1256
year is 2001, gauge num is 1257
year is 2001, gauge num is 1258
year is 2001, gauge num is 1259
year is 2001, gauge num is 1260
year is 2001, gauge num is 1261
year is 2001, gauge num is 1262
year is 2001, gauge num is 1263
year is 2001, gauge num is 1264
year is 2001, gauge num is 1265
year is 2001, gauge num is 1266
year is 2001, gauge num is 1267
year is 2001, gauge num is 1268
year is 2001, gauge num is 1269
year is 2001, gauge num is 1270
year is 2001, gauge num is 1271
year is 2001, gauge num is 1272
year is 2001, gauge num is 1273
year is 2001, gauge num is 1274
year is 2001, gauge num is 1275
year is 2001, gauge num is 1276
year is 2001, gauge num is 1277
year is 2001, gauge num is 1278
year is 2001, gauge num is 1279
year is 2001, gauge num is 1280
year is 

...

(210, 279)
Time to load data is 11.82 seconds
Finding the AMAX for 0.5hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.06559753417969
Finding the AMAX for 1hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.06559753417969
Finding the AMAX for 2hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.06559753417969
Finding the AMAX for 3hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.07260131835938
Finding the AMAX for 6hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.07260131835938
Finding the AMAX for 12hr events for gauge 713 in year 2007
Event doesnt contain NAN, total event precip is 106.41349792480469
Finding the AMAX for 24hr events for gauge 713 in year 2007
2 events
Event doesnt contain NAN, total event precip is 106.4450912475586
year is 2007, gauge num is 714
(220, 281)
Time to load data 

year is 2007, gauge num is 722
(224, 273)
Time to load data is 11.64 seconds
Finding the AMAX for 0.5hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 38.2416877746582
Finding the AMAX for 1hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 38.2416877746582
Finding the AMAX for 2hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 38.2416877746582
Finding the AMAX for 3hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 38.2416877746582
Finding the AMAX for 6hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 38.2416877746582
Finding the AMAX for 12hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 70.8891372680664
Finding the AMAX for 24hr events for gauge 722 in year 2007
Event doesnt contain NAN, total event precip is 58.17725372314453
year is 2007, gauge num is 723
(219, 274)
Time to 

(223, 265)
Time to load data is 12.31 seconds
Finding the AMAX for 0.5hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.21660614013672
Finding the AMAX for 1hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.21660614013672
Finding the AMAX for 2hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.21660614013672
Finding the AMAX for 3hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.21660614013672
Finding the AMAX for 6hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.43893432617188
Finding the AMAX for 12hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 65.43893432617188
Finding the AMAX for 24hr events for gauge 731 in year 2007
Event doesnt contain NAN, total event precip is 68.2276840209961
year is 2007, gauge num is 732
(227, 265)
Time to load data is 15.64 seconds

(228, 267)
Time to load data is 12.97 seconds
Finding the AMAX for 0.5hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 63.51727294921875
Finding the AMAX for 1hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 63.51727294921875
Finding the AMAX for 2hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 63.51727294921875
Finding the AMAX for 3hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 63.51727294921875
Finding the AMAX for 6hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 63.786502838134766
Finding the AMAX for 12hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 126.50603485107422
Finding the AMAX for 24hr events for gauge 740 in year 2007
Event doesnt contain NAN, total event precip is 126.50603485107422
year is 2007, gauge num is 741
(229, 267)
Time to load data is 11.75 sec

Event doesnt contain NAN, total event precip is 64.15816497802734
year is 2007, gauge num is 749
(233, 263)
Time to load data is 14.71 seconds
Finding the AMAX for 0.5hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 54.1605224609375
Finding the AMAX for 1hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 54.1605224609375
Finding the AMAX for 2hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 54.1605224609375
Finding the AMAX for 3hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 54.1605224609375
Finding the AMAX for 6hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 56.40823745727539
Finding the AMAX for 12hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 56.55311965942383
Finding the AMAX for 24hr events for gauge 749 in year 2007
Event doesnt contain NAN, total event precip is 